In [ ]:
# | default_exp client.__init__

# Client

> Web client to make requests to New Relic API

## Imports

Python modules

In [ ]:
# | export
import json
import os
import pathlib
from typing import Any, Dict, Union

Third-party modules

In [ ]:
# | export
import dotenv
from requests import Response, Session
from sgqlc.types import Schema

`newrelic_sb_sdk` module

In [ ]:
# | export
from newrelic_sb_sdk.graphql import nerdgraph
from newrelic_sb_sdk.graphql.objects import RootMutationType, RootQueryType
from newrelic_sb_sdk.utils.query import build_query
from newrelic_sb_sdk.version import VERSION

## Functions

In [ ]:
# | export


def get_new_relic_user_key_from_env(env_file_name: Union[str, None] = None) -> str:
    """Recovery new relic credentials from environmentn variables."""

    if env_file_name is not None:
        env_file = pathlib.Path(env_file_name)

        if env_file.exists():
            dotenv.load_dotenv(env_file)

    new_relic_user_key = os.environ.get("NEW_RELIC_USER_KEY", None)

    if new_relic_user_key is None:
        raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

    return new_relic_user_key

## Classes

In [ ]:
# | export


class NewRelicGqlClient(Session):
    """Client for New Relic GraphQL API."""

    _url: str = "https://api.newrelic.com/graphql"
    _schema: Schema = nerdgraph

    def __init__(self, *, new_relic_user_key: str):
        super().__init__()

        self.headers.update(
            {
                "Accept": "application/json",
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
                "User-Agent": f"newrelic-sb-sdk/{self._get_version()}",
            }
        )

        self._setup_schema()

    @staticmethod
    def _get_version():
        return ".".join(VERSION)

    def _setup_schema(self):
        self._schema.query_type = RootQueryType
        self._schema.mutation_type = RootMutationType

    def execute(
        self, query: str, *, variables: Union[Dict[str, Any], None] = None, **kwargs
    ) -> Response:
        data = json.dumps(
            {
                "query": query,
                "variables": variables,
            },
        )
        return self.post(self._url, data=data, **kwargs)

    @staticmethod
    def build_query(
        template: str, *, params: Union[Dict[str, Any], None] = None
    ) -> str:
        return build_query(template, params=params)

    @property
    def schema(self) -> Schema:
        return self._schema

In [ ]:
# | export


class NewRelicRestClient(Session):
    """Client for New Relic Rest API."""

    url: str = "https://api.newrelic.com/v2/"

    def __init__(self, *, new_relic_user_key: str):
        super().__init__()

        self.headers.update(
            {
                "Accept": "application/json",
                "Content-Type": "application/json",
                "Api-Key": new_relic_user_key,
                "User-Agent": f"newrelic-sb-sdk/{self._get_version()}",
            }
        )

    @staticmethod
    def _get_version():
        return ".".join(VERSION)